In [2]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
import datetime

# Processing Units logs
log_device_placement = True

# Num of multiplications to perform
n = 10

# Create a graph to store results
c = []

def matpow(M, n):
    if n < 1: #Abstract cases where n < 1
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

def computation(x, n):
    return matpow(x, n)
    
with tf.device('/device:CPU:0'):
    a0 = tf.placeholder(tf.float32, [1000, 1000])
    res = computation(a0, n)
    c.append(res)

with tf.device('/device:CPU:1'):
    a1 = tf.placeholder(tf.float32, [1000, 1000])
    res = computation(a1, n)
    c.append(res)

with tf.device('/device:CPU:2'):
    sum = tf.add_n(c)

config = tf.ConfigProto(
    log_device_placement=log_device_placement,
    device_count={"CPU": 3},
    inter_op_parallelism_threads=1,
    intra_op_parallelism_threads=1
)

# Create random large matrix
A0 = np.random.rand(1000, 1000).astype('float32')
A1 = np.random.rand(1000, 1000).astype('float32')

t1_2 = datetime.datetime.now()
with tf.Session(config=config) as sess:
    # Run the op.
    sess.run(sum, {a0:A0, a1:A1})
t2_2 = datetime.datetime.now()


# print("Single GPU computation time: " + str(t2_1-t1_1))
print("Multi GPU computation time: " + str(t2_2-t1_2))

Multi GPU computation time: 0:00:01.557971
